# 📘 Step 0: 환경 설정 및 Baseline RAG Agent 구축

## 목표
- MLflow 설치 및 기본 환경 설정
- LangGraph 기반 RAG Agent 구현 (MLflow 없이)
- 벤치마크용 baseline 성능 확인

## 학습 포인트
- LangGraph State와 Node 개념
- RAG 파이프라인 기본 구조
- 성능 측정 기준 설정

## 1. 환경 설정 및 패키지 설치

### 1.1 필요한 패키지 설치

In [ ]:
# 주석을 해제하고 실행하세요
# !pip install mlflow==2.21.3
# !pip install langchain==0.3.12
# !pip install langchain-openai==0.2.14
# !pip install langchain-community==0.3.12
# !pip install langgraph==0.2.63
# !pip install faiss-cpu==1.9.0
# !pip install python-dotenv==1.0.1
# !pip install tiktoken==0.8.0

### 1.2 환경 변수 로드

In [21]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

True

### 1.3 필요한 라이브러리 임포트

In [ ]:
import time
from typing import TypedDict, List, Annotated
from operator import add

from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

from langgraph.graph import StateGraph, START, END

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


## 2. 샘플 문서 데이터 준비

RAG 시스템 테스트를 위한 샘플 문서를 생성합니다.

In [3]:
# 샘플 문서 생성 (실제 프로젝트에서는 파일이나 데이터베이스에서 로드)
sample_documents = [
    """
    RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.
    RAG는 외부 지식 베이스에서 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.
    이 방법은 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.
    """,
    """
    Vector Store는 문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스입니다.
    FAISS, Chroma, Pinecone 등이 대표적인 Vector Store입니다.
    Vector Store는 유사도 검색(Similarity Search)을 통해 쿼리와 가장 관련성 높은 문서를 빠르게 찾아냅니다.
    """,
    """
    LangGraph는 LangChain 위에 구축된 상태 기반 그래프 프레임워크입니다.
    복잡한 AI 에이전트 워크플로우를 명확한 상태와 노드로 표현할 수 있습니다.
    각 노드는 특정 작업을 수행하고, 엣지는 노드 간의 전환 조건을 정의합니다.
    """,
    """
    Embeddings는 텍스트를 고차원 벡터 공간의 점으로 표현하는 기술입니다.
    OpenAI의 text-embedding-3-small, text-embedding-3-large 모델이 널리 사용됩니다.
    좋은 임베딩은 의미적으로 유사한 텍스트를 벡터 공간에서 가까이 위치시킵니다.
    """,
    """
    Chunking은 긴 문서를 작은 조각으로 나누는 과정입니다.
    적절한 chunk 크기는 검색 정확도와 컨텍스트 양의 균형을 맞추는 데 중요합니다.
    일반적으로 512~1024 토큰 크기의 chunk가 많이 사용되며, 문장 경계를 고려한 분할이 효과적입니다.
    """,
]

print(f"✅ {len(sample_documents)}개의 샘플 문서 생성 완료")

✅ 5개의 샘플 문서 생성 완료


## 3. Vector Store 구축

문서를 임베딩하고 FAISS Vector Store에 저장합니다.

In [ ]:
# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

# 문서를 Document 객체로 변환
documents = [Document(page_content=doc.strip()) for doc in sample_documents]

# 문서 분할
split_documents = text_splitter.split_documents(documents)
print(f"📄 총 {len(split_documents)}개의 chunk 생성")

# 임베딩 모델 초기화
embeddings = BedrockEmbeddings(
    model_id=os.environ["AWS_EMD_MODEL_ID"],
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name=os.environ["AWS_REGION"],
)

# Vector Store 생성
print("🔄 Vector Store 생성 중...")
vector_store = FAISS.from_documents(
    documents=split_documents,
    embedding=embeddings
)

print("✅ Vector Store 생성 완료")

📄 총 5개의 chunk 생성
🔄 Vector Store 생성 중...
✅ Vector Store 생성 완료


## 4. LangGraph State 정의

RAG Agent의 상태를 관리할 TypedDict를 정의합니다.

In [5]:
class RAGState(TypedDict):
    """RAG Agent의 상태를 정의하는 클래스"""
    query: str                           # 사용자 질문
    retrieved_documents: List[Document]  # 검색된 문서들
    context: str                         # 문서를 텍스트로 결합한 컨텍스트
    answer: str                          # 최종 생성된 답변
    metadata: dict                       # 추가 메타데이터 (실행 시간 등)

print("✅ RAGState 정의 완료")

✅ RAGState 정의 완료


## 5. RAG Agent Nodes 구현

### 5.1 Retriever Node

In [6]:
def retriever_node(state: RAGState) -> RAGState:
    """
    Vector Store에서 관련 문서를 검색하는 노드
    """
    print(f"\n🔍 [Retriever] 검색 시작: '{state['query']}'")
    
    start_time = time.time()
    
    # Vector Store에서 유사도 검색 (top_k=3)
    retrieved_docs = vector_store.similarity_search(
        query=state["query"],
        k=3
    )
    
    retrieval_time = time.time() - start_time
    
    # 상태 업데이트
    state["retrieved_documents"] = retrieved_docs
    state["metadata"]["retrieval_time"] = retrieval_time
    state["metadata"]["num_retrieved_docs"] = len(retrieved_docs)
    
    print(f"✅ [Retriever] {len(retrieved_docs)}개 문서 검색 완료 ({retrieval_time:.3f}초)")
    
    # 검색된 문서 미리보기
    for i, doc in enumerate(retrieved_docs, 1):
        preview = doc.page_content[:100].replace("\n", " ")
        print(f"  📄 문서 {i}: {preview}...")
    
    return state

print("✅ Retriever Node 구현 완료")

✅ Retriever Node 구현 완료


### 5.2 Generator Node

In [12]:
def generator_node(state: RAGState) -> RAGState:
    """
    검색된 문서를 기반으로 답변을 생성하는 노드
    """
    print("\n✍️ [Generator] 답변 생성 시작")
    
    start_time = time.time()
    
    # 검색된 문서를 하나의 컨텍스트로 결합
    context = "\n\n".join([
        f"[문서 {i+1}]\n{doc.page_content}"
        for i, doc in enumerate(state["retrieved_documents"])
    ])
    state["context"] = context
    
    # 프롬프트 템플릿 정의
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", """
당신은 질문에 답변하는 친절한 AI 어시스턴트입니다.
제공된 컨텍스트를 기반으로 정확하고 간결한 답변을 제공하세요.
컨텍스트에 정보가 없으면 "제공된 문서에서 관련 정보를 찾을 수 없습니다"라고 답변하세요.
"""),
        ("human", """
컨텍스트:
{context}

질문: {query}

답변:
""")
    ])
    
    # LLM 초기화
    llm = ChatBedrock(
        model_id=os.environ["AWS_MODEL_ID"],
        aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
        region_name=os.environ["AWS_REGION"],
        temperature=0.0,
    )
    
    # 프롬프트 생성 및 LLM 호출
    messages = prompt_template.format_messages(
        context=context,
        query=state["query"]
    )
    
    response = llm.invoke(messages)
    answer = response.content
    
    generation_time = time.time() - start_time
    
    # 상태 업데이트
    state["answer"] = answer
    state["metadata"]["generation_time"] = generation_time
    state["metadata"]["llm_model"] = os.environ["AWS_MODEL_ID"]
    
    print(f"✅ [Generator] 답변 생성 완료 ({generation_time:.3f}초)")
    
    return state

print("✅ Generator Node 구현 완료")

✅ Generator Node 구현 완료


## 6. LangGraph 구성

Retriever와 Generator 노드를 연결하여 RAG 그래프를 구성합니다.

In [13]:
# StateGraph 초기화
workflow = StateGraph(RAGState)

# 노드 추가
workflow.add_node("retriever", retriever_node)
workflow.add_node("generator", generator_node)

# 엣지 연결
workflow.add_edge(START, "retriever")      # 시작 -> Retriever
workflow.add_edge("retriever", "generator") # Retriever -> Generator
workflow.add_edge("generator", END)         # Generator -> 종료

# 그래프 컴파일
rag_agent = workflow.compile()

print("✅ RAG 그래프 구성 완료")
print("\n📊 그래프 구조:")
print("  START → Retriever → Generator → END")

✅ RAG 그래프 구성 완료

📊 그래프 구조:
  START → Retriever → Generator → END


## 7. RAG Agent 실행 및 테스트

### 7.1 테스트 쿼리 정의

In [14]:
# 테스트용 질문들
test_queries = [
    "RAG가 무엇인가요?",
    "Vector Store의 역할은 무엇인가요?",
    "LangGraph는 어떤 프레임워크인가요?",
    "Embedding과 Chunking에 대해 설명해주세요.",
]

print(f"✅ {len(test_queries)}개의 테스트 쿼리 준비 완료")

✅ 4개의 테스트 쿼리 준비 완료


### 7.2 단일 쿼리 테스트

In [15]:
def run_rag_agent(query: str) -> dict:
    """
    RAG Graph를 실행하고 결과를 반환하는 함수
    """
    print("=" * 80)
    print(f"🚀 RAG 실행")
    print("=" * 80)
    
    # 초기 상태 설정
    initial_state = {
        "query": query,
        "retrieved_documents": [],
        "context": "",
        "answer": "",
        "metadata": {}
    }
    
    # 전체 실행 시간 측정
    overall_start = time.time()
    
    # RAG Agent 실행
    final_state = rag_agent.invoke(initial_state)
    
    overall_time = time.time() - overall_start
    final_state["metadata"]["overall_time"] = overall_time
    
    return final_state


# 첫 번째 쿼리로 테스트
result = run_rag_agent(test_queries[0])

🚀 RAG 실행

🔍 [Retriever] 검색 시작: 'RAG가 무엇인가요?'
✅ [Retriever] 3개 문서 검색 완료 (0.102초)
  📄 문서 1: RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.     RAG는 외부 지식 베이스에서 관련 정보를 검...
  📄 문서 2: LangGraph는 LangChain 위에 구축된 상태 기반 그래프 프레임워크입니다.     복잡한 AI 에이전트 워크플로우를 명확한 상태와 노드로 표현할 수 있습니다.     각...
  📄 문서 3: Chunking은 긴 문서를 작은 조각으로 나누는 과정입니다.     적절한 chunk 크기는 검색 정확도와 컨텍스트 양의 균형을 맞추는 데 중요합니다.     일반적으로 512~...

✍️ [Generator] 답변 생성 시작
✅ [Generator] 답변 생성 완료 (4.579초)


### 7.3 결과 출력

In [17]:
def print_result(result: dict):
    """
    RAG 실행 결과를 보기 좋게 출력
    """
    print("\n" + "=" * 80)
    print("📋 실행 결과")
    print("=" * 80)
    
    print(f"\n❓ 질문: {result['query']}")
    print(f"\n💬 답변:\n{result['answer']}")
    
    print("\n" + "-" * 80)
    print("📊 성능 메트릭")
    print("-" * 80)
    
    metadata = result['metadata']
    print(f"  • 전체 실행 시간: {metadata['overall_time']:.3f}초")
    print(f"  • 검색 시간: {metadata['retrieval_time']:.3f}초")
    print(f"  • 생성 시간: {metadata['generation_time']:.3f}초")
    print(f"  • 검색된 문서 수: {metadata['num_retrieved_docs']}개")
    print(f"  • 사용된 LLM: {metadata['llm_model']}")
    
    print("\n" + "-" * 80)
    print("📄 검색된 문서 (요약)")
    print("-" * 80)
    
    for i, doc in enumerate(result['retrieved_documents'], 1):
        preview = doc.page_content[:150].replace("\n", " ")
        print(f"\n  [{i}] {preview}...")
    
    print("\n" + "=" * 80)


# 결과 출력
print_result(result)


📋 실행 결과

❓ 질문: RAG가 무엇인가요?

💬 답변:
RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다. 

RAG의 주요 특징은 다음과 같습니다:

1. **외부 지식 검색**: 외부 지식 베이스에서 관련 정보를 검색합니다.

2. **정확한 답변 생성**: 검색된 정보를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.

3. **최신 정보 활용**: 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.

즉, RAG는 LLM이 자체 학습 데이터에만 의존하지 않고, 필요한 정보를 외부에서 검색하여 더 신뢰할 수 있는 답변을 제공하는 방식입니다.

--------------------------------------------------------------------------------
📊 성능 메트릭
--------------------------------------------------------------------------------
  • 전체 실행 시간: 4.684초
  • 검색 시간: 0.102초
  • 생성 시간: 4.579초
  • 검색된 문서 수: 3개
  • 사용된 LLM: global.anthropic.claude-sonnet-4-5-20250929-v1:0

--------------------------------------------------------------------------------
📄 검색된 문서 (요약)
--------------------------------------------------------------------------------

  [1] RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.     RAG는 외부 지식 베이스에서 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 사실에 기반한 답변을 생

### 7.4 여러 쿼리 배치 테스트

In [18]:
# 모든 테스트 쿼리 실행
all_results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n\n{'='*80}")
    print(f"테스트 {i}/{len(test_queries)}")
    print(f"{'='*80}")
    
    result = run_rag_agent(query)
    all_results.append(result)
    
    # 간단한 결과만 출력
    print(f"\n✅ 완료 - 전체 시간: {result['metadata']['overall_time']:.3f}초")
    print(f"💬 답변 미리보기: {result['answer'][:100]}...")



테스트 1/4
🚀 RAG 실행

🔍 [Retriever] 검색 시작: 'RAG가 무엇인가요?'
✅ [Retriever] 3개 문서 검색 완료 (0.100초)
  📄 문서 1: RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.     RAG는 외부 지식 베이스에서 관련 정보를 검...
  📄 문서 2: LangGraph는 LangChain 위에 구축된 상태 기반 그래프 프레임워크입니다.     복잡한 AI 에이전트 워크플로우를 명확한 상태와 노드로 표현할 수 있습니다.     각...
  📄 문서 3: Chunking은 긴 문서를 작은 조각으로 나누는 과정입니다.     적절한 chunk 크기는 검색 정확도와 컨텍스트 양의 균형을 맞추는 데 중요합니다.     일반적으로 512~...

✍️ [Generator] 답변 생성 시작
✅ [Generator] 답변 생성 완료 (6.690초)

✅ 완료 - 전체 시간: 6.793초
💬 답변 미리보기: RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다. 

RAG의 주요 특징은 다음과 같습니다:

1. *...


테스트 2/4
🚀 RAG 실행

🔍 [Retriever] 검색 시작: 'Vector Store의 역할은 무엇인가요?'
✅ [Retriever] 3개 문서 검색 완료 (0.112초)
  📄 문서 1: Vector Store는 문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스입니다.     FAISS, Chroma, Pinecone 등이 대표적인 Vector Store입니다. ...
  📄 문서 2: Embeddings는 텍스트를 고차원 벡터 공간의 점으로 표현하는 기술입니다.     OpenAI의 text-embedding-3-small, text-embedding-3-lar...
  📄 문서 3: RAG (Retrieval-Augmented Generati

## 8. 성능 벤치마크 요약

모든 테스트 결과를 요약하여 baseline 성능을 확인합니다.

In [22]:
import statistics

# 성능 메트릭 수집
overall_times = [r['metadata']['overall_time'] for r in all_results]
retrieval_times = [r['metadata']['retrieval_time'] for r in all_results]
generation_times = [r['metadata']['generation_time'] for r in all_results]

print("\n" + "=" * 80)
print("📊 Baseline RAG 성능 벤치마크")
print("=" * 80)

print(f"\n📈 테스트 수행: {len(all_results)}개 쿼리")
print(f"\n⏱️  평균 실행 시간:")
print(f"  • 전체: {statistics.mean(overall_times):.3f}초 (± {statistics.stdev(overall_times):.3f})")
print(f"  • 검색: {statistics.mean(retrieval_times):.3f}초 (± {statistics.stdev(retrieval_times):.3f})")
print(f"  • 생성: {statistics.mean(generation_times):.3f}초 (± {statistics.stdev(generation_times):.3f})")

print(f"\n⚡ 최소/최대 실행 시간:")
print(f"  • 최소: {min(overall_times):.3f}초")
print(f"  • 최대: {max(overall_times):.3f}초")

print(f"\n🎯 설정:")
print(f"  • Embedding Model: {os.environ["AWS_EMD_MODEL_ID"]}")
print(f"  • LLM Model: {result['metadata']['llm_model']}")
print(f"  • Top-K: 3")
print(f"  • Chunk Size: 512")

print("\n" + "=" * 80)


📊 Baseline RAG 성능 벤치마크

📈 테스트 수행: 4개 쿼리

⏱️  평균 실행 시간:
  • 전체: 5.035초 (± 1.517)
  • 검색: 0.101초 (± 0.008)
  • 생성: 4.930초 (± 1.519)

⚡ 최소/최대 실행 시간:
  • 최소: 3.487초
  • 최대: 6.793초

🎯 설정:
  • Embedding Model: amazon.titan-embed-text-v2:0
  • LLM Model: global.anthropic.claude-sonnet-4-5-20250929-v1:0
  • Top-K: 3
  • Chunk Size: 512



## 9. 상세 결과 확인 (선택)

특정 쿼리의 전체 결과를 자세히 확인할 수 있습니다.

In [23]:
# 첫 번째 결과 상세 출력
print_result(all_results[0])


📋 실행 결과

❓ 질문: RAG가 무엇인가요?

💬 답변:
RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다. 

RAG의 주요 특징은 다음과 같습니다:

1. **외부 지식 검색**: 외부 지식 베이스에서 관련 정보를 검색합니다.

2. **정확한 답변 생성**: 검색된 정보를 바탕으로 더 정확하고 사실에 기반한 답변을 생성합니다.

3. **최신 정보 활용**: 모델이 학습하지 않은 최신 정보나 특정 도메인 지식을 활용할 수 있게 해줍니다.

즉, RAG는 LLM이 자체 학습 데이터에만 의존하지 않고, 필요한 정보를 외부에서 검색하여 더 신뢰할 수 있는 답변을 제공하는 방식입니다.

--------------------------------------------------------------------------------
📊 성능 메트릭
--------------------------------------------------------------------------------
  • 전체 실행 시간: 6.793초
  • 검색 시간: 0.100초
  • 생성 시간: 6.690초
  • 검색된 문서 수: 3개
  • 사용된 LLM: global.anthropic.claude-sonnet-4-5-20250929-v1:0

--------------------------------------------------------------------------------
📄 검색된 문서 (요약)
--------------------------------------------------------------------------------

  [1] RAG (Retrieval-Augmented Generation)는 대규모 언어 모델(LLM)의 능력을 향상시키는 기술입니다.     RAG는 외부 지식 베이스에서 관련 정보를 검색하고, 이를 바탕으로 더 정확하고 사실에 기반한 답변을 생

## 10. 대화형 테스트 (선택)

직접 질문을 입력하여 RAG Agent를 테스트할 수 있습니다.

In [24]:
# 커스텀 쿼리 테스트
custom_query = "Embedding은 무엇이고 왜 중요한가요?"  # 원하는 질문으로 변경

custom_result = run_rag_agent(custom_query)
print_result(custom_result)

🚀 RAG 실행

🔍 [Retriever] 검색 시작: 'Embedding은 무엇이고 왜 중요한가요?'
✅ [Retriever] 3개 문서 검색 완료 (0.104초)
  📄 문서 1: Embeddings는 텍스트를 고차원 벡터 공간의 점으로 표현하는 기술입니다.     OpenAI의 text-embedding-3-small, text-embedding-3-lar...
  📄 문서 2: Vector Store는 문서의 임베딩 벡터를 저장하고 검색하는 데이터베이스입니다.     FAISS, Chroma, Pinecone 등이 대표적인 Vector Store입니다. ...
  📄 문서 3: Chunking은 긴 문서를 작은 조각으로 나누는 과정입니다.     적절한 chunk 크기는 검색 정확도와 컨텍스트 양의 균형을 맞추는 데 중요합니다.     일반적으로 512~...

✍️ [Generator] 답변 생성 시작
✅ [Generator] 답변 생성 완료 (5.159초)

📋 실행 결과

❓ 질문: Embedding은 무엇이고 왜 중요한가요?

💬 답변:
Embedding은 텍스트를 고차원 벡터 공간의 점으로 표현하는 기술입니다.

**Embedding이 중요한 이유:**

1. **의미적 유사성 표현**: 좋은 임베딩은 의미적으로 유사한 텍스트를 벡터 공간에서 가까이 위치시켜, 텍스트 간의 의미적 관계를 수치적으로 표현할 수 있습니다.

2. **효율적인 검색**: Vector Store와 결합하여 유사도 검색(Similarity Search)을 가능하게 하며, 쿼리와 가장 관련성 높은 문서를 빠르게 찾아낼 수 있습니다.

3. **RAG 시스템의 핵심**: 문서를 임베딩으로 변환하여 저장하고, 사용자 질문과 유사한 문서를 검색하는 RAG(Retrieval-Augmented Generation) 시스템의 기반 기술입니다.

현재 OpenAI의 text-embedding-3-small, text-embedding-3-large 모델이 널리 사용되고 있습니다.

-

## 11. 요약 및 다음 단계

### ✅ 완료한 작업
1. MLflow 및 LangGraph 환경 설정
2. 샘플 문서로 Vector Store 구축
3. LangGraph 기반 RAG Agent 구현
4. Baseline 성능 측정 및 벤치마크

### 📊 Baseline 성능 지표
- 평균 응답 시간: ~X초
- 검색 시간: ~X초
- 생성 시간: ~X초
- 문서 검색 수: 3개

### 🎯 다음 단계 (Step 1)
다음 노트북에서는 **MLflow Tracking**을 도입하여:
- 실험 결과를 자동으로 기록
- Parameters, Metrics, Artifacts 관리
- MLflow UI에서 시각화
- 여러 실험 간 비교 분석

→ `01_mlflow_tracking_basics.ipynb`로 계속하세요!

## 📚 참고 자료

- [MLflow Documentation](https://mlflow.org/docs/latest/)
- [LangChain Documentation](https://python.langchain.com/docs/)
- [LangGraph Tutorial](https://langchain-ai.github.io/langgraph/)
- [FAISS Documentation](https://faiss.ai/)
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)